In [ ]:
def read_code_from_file(file_path: str) -> str:
    with open(file_path, 'r') as file:
        code = file.read()
    return code

def write_code_to_file(file_path: str, code: str):
    with open(file_path, 'w') as file:
        file.write(code)

In [ ]:
from typing import Annotated
from langchain_experimental.utilities import PythonREPL
from langchain_core.tools import tool

repl = PythonREPL()

@tool
def python_repl(
    code: Annotated[str, "filename to read the code from"]
):
    """Use this to execute python code read from a file. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""

    try:
        result = repl.run(code)
        print("RESULT CODE EXECUTION:", result)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    return f"Successfully executed:\n```python\n{code}\n```\nStdout: {result}"



In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage


model = ChatOpenAI()
model_with_tools = model.bind_tools([python_repl])

In [ ]:
messages = [
    SystemMessage(content=""" You have got the task to execute code. Use the pyhon_repl tool to execute it.
        If the code produced an error just return 'False'. If it worked, return 'True'"""),
    HumanMessage(content="testscript.py")
    ]

ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage

for tool_call in ai_msg.tool_calls:
    selected_tool = {"python_repl": python_repl}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

model_with_tools.invoke(messages)

### Working with Agents

In [ ]:
from typing import TypedDict

class AgentState(TypedDict):
    message: str
    error: bool
    error_message: str
    file_path: str
    code: str
    iterations: int

In [ ]:
def identify_filepath(state: AgentState):
    message = state["message"]
    model = ChatOpenAI()
    messages = [SystemMessage(content="""Your task is to evaluate the userinput and extract the filename he provided.
                              ONLY return the last filename, nothing else!"""),
    HumanMessage(content=message)]
    result = model.invoke(messages)
    state["file_path"] = result.content
    print(f"STATE bei Identifier filepath: {state}")
    return state


def execute_code_with_model(state: AgentState):

    code = read_code_from_file(state["file_path"])
    print("CODE:", code)

    model = ChatOpenAI()
    model_with_tools = model.bind_tools([python_repl])

    messages = [
        SystemMessage(content=""" You have got the task to execute code. Use the python_repl tool to execute it.
            If the code produced an error just return 'True'. If it worked, return 'False'"""),
        HumanMessage(content=code)
    ]

    ai_msg = model_with_tools.invoke(messages)
    messages.append(ai_msg)

    for tool_call in ai_msg.tool_calls:
        selected_tool = {"python_repl": python_repl}[tool_call["name"].lower()]
        tool_output = selected_tool.invoke(tool_call["args"])
        state["code"] = tool_output
        messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

    result = model_with_tools.invoke(messages)
    print("EXECUTE_RESULT", result.content)
    state["error"] = result.content
    print(f"STATE bei read_code_from_file: {state}")
    return state


def rewrite_code(state: AgentState):

    code = state["code"]
    state["iterations"] += 1
    model = ChatOpenAI()
    messages = [
        SystemMessage(content="You can to analyze the following code provided in the usermessage. Your task is to fix that code and provide the user the correct new code."),
        HumanMessage(content=code)
    ]
    ai_msg = model.invoke(messages)
    write_code_to_file(file_path=f'{state["file_path"]}', code=ai_msg.content)
    state["code"] = ai_msg.content
    print(f"STATE bei rewrite_code: {state}")
    return state


def next_step(state: AgentState):
    if bool(state["error"]) == True:
        return "error"
    elif state["iterations"] > 5:
        print("Max Iterations done.... Exit Agent")
        return "max_iterations"
    else:
        return "ok"

In [ ]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(AgentState)

workflow.add_node("identify_filepath", identify_filepath)
workflow.add_node("execute_code_with_model", execute_code_with_model)
workflow.add_node("rewrite_code", rewrite_code)

workflow.set_entry_point("identify_filepath")
workflow.add_edge("identify_filepath", "execute_code_with_model")

workflow.add_conditional_edges(
    "execute_code_with_model",
    next_step,
    {
        "error": "rewrite_code",
        "ok": END,
        "max_iterations": END
    },
)
workflow.add_edge("rewrite_code", "execute_code_with_model")
app = workflow.compile()

In [ ]:
from IPython.display import Image, display

display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
app.invoke({"message": "Please analyze the testscript.py file", "error": "", "error_message": "", "file_path": "", "code": "", "iterations": 0})